In [29]:
import os
import os.path
import random
import pickle
import zipfile
import numpy as np
from tqdm import tqdm
import pandas as pd
import tensorflow as tf      #the progress bar
import en_core_web_sm as en  #from the spaCy library, https://spacy.io/usage/

In [30]:
filename = "../input/quora-insincere-questions-classification/train.csv"

In [31]:
# enb_path = 'input/embeddings/wiki-news-300d-1M/' + 'wiki-news-300d-1M.vec'
# enb_path = 'input/embeddings/GoogleNews-vectors-negative300/'
# enb_path = 'input/embeddings/paragram_300_sl999/'
enb_path = "../input/quora-insincere-questions-classification/embeddings/glove.840B.300d/" + "glove.840B.300d.txt"

In [32]:
#global variables to save memory
# _t is for tokenized
embedings = []      #np array of enbedings
vocab = []          #the vocabulary from the enbeding file
questions_t = []    #every question tokenized in an array
text_t = []         #all the questions concatenated and tokenized
word_to_index = {}
index_to_word = {}

In [33]:
def tokenize_questions(text,nr_to_delete = 0):
    nlp = en.load()      #load the tokenizer
    
    global questions_t
    global vocab               #every unique word in the appearance order  
    global text_t      #every token in the original order
    
    text_t = [] 
    vocab = [] 
    questions_t = []
    
    batches = text.tolist()
    
    del text  #delete the original text so we save memory
    bar  = tqdm(total = len(batches))
    
    for batch in batches:  #tqdm is the progress bar
        tokens = nlp(batch)
        batch_tokenized = []
        for token in tokens:
            word = token.string.strip()
            text_t.append(word)
            batch_tokenized.append(word)
        questions_t.append(batch_tokenized)
        bar.update(1)
    bar.close()

In [34]:
train_df = pd.read_csv(filename)    #test_t is for now a dataFrame

In [45]:
if not os.path.isfile("../output/questions_t"):
    tokenize_questions( train_df.values[:,1] )
    pickle.dump(questions_t, open("../output/batches_tokenized",'wb') )
    pickle.dump(text_t, open("../output/text_tokenized",'wb') )
else:
    print("Loading data...")
    questions_t = pickle.load( open( "../output/questions_t", "rb" ) )
    text_t = pickle.load( open( "../output/text_t", "rb" ) )
    print("Nr of questions: ",len(questions_t))
    print("Nr of tokens:    ",len(text_t))

Loading data...
Nr of questions:  1306122
Nr of tokens:     19020768


In [36]:
def load_embedings(enb_path):
    global embedings
    global vocab
    
    embedings = []
    vocab = []
    
    print("Loading enbedings...")
    
    num_lines = sum(1 for line in open(enb_path))
    bar  = tqdm(total = num_lines)
    
    with open(enb_path, "r") as ins:
        array = ins.readline()[:-1].split(' ')
        vocab = [str(array[0])]
        embedings = [np.array(array[1:]).astype('float32')]
        
        for line in ins:
            bar.update(1)
            array = line[:-1].split(' ')
            vocab.append(str(array[0]))
            embedings.append(np.array(array[1:]).astype('float32'))      
        bar.close()
        embedings = np.stack(embedings)
    print("Done!")

In [37]:
if not os.path.isfile("../output/embedings"):
    load_embedings(enb_path)
    pickle.dump(embedings,open( "../output/embedings",'wb'))
    pickle.dump(vocab,open( "../output/vocabulary",'wb'))
else:
    print("Loading enbedings...")
    embedings = pickle.load( open( "../output/embedings", "rb" ) )
    vocab = pickle.load( open( "../output/vocabulary", "rb" ) )
print('Embedings size:  ',embedings.shape)
print('Vocabulary size: ', len(vocab))


Loading enbedings...
Embedings size:   (2196017, 300)
Vocabulary size:  2196017


In [55]:
def tokens_to_index():
    global word_to_index
    global index_to_word
    global questions_t
    global vocab
    
    word_to_index = {}
    index_to_word = {}
    
    for idx, word in enumerate(vocab): #create the two dictionaries
        word_to_index[word] = idx
        index_to_word[idx] = word
        
    for batch in questions_t: #chage the input from strings to ids
        if isinstance(batch, int):
            print(batch)
        for idx,token in enumerate(batch): #replace the text with the word indexes
             batch[idx] = word_to_index.get(token, "UNK")
tokens_to_index()

In [70]:
questions_t[0]

[255, 127, 9833, 62681, 118, 58, 7599, 28, 6, 1990, 7, 2, 10310, 37]